In [23]:
import copy, sys
import random, numpy as np

sys.path.append("../../")
from importlib import reload

import utils.display.tables as tables
import inferring.InferringDFA as InferringDFA
import utils.runLearnLib.RunLearnLib as RunLearnLib
import utils.automata.DFA
import utils.advice_systems.SRS as SRS

reload(InferringDFA)
reload(RunLearnLib)
reload(SRS)
reload(utils.automata.DFA)
reload(tables)

from utils.runLearnLib.RunLearnLib import RunLearnLib
from inferring.InferringDFA import InferringDFA
from utils.automata.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.display.tables import Tables

In [24]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [25]:
class Record:

    def __init__(self, dfa_sz, common, lstar, lstar_as, target=None):
        self.dfa_sz = dfa_sz
        self.common = common
        self.lstar = lstar
        self.lstar_as = lstar_as
        self.target = target
    def print_record(self):
        print(
            f"|dfa_sz| = {self.dfa_sz}, common letters = {int(self.common*100)}% eq = {self.lstar[1]}, eq_with_advice = {self.lstar_as[1]}"
        )
    def print_target(self):
        print(self.target.print_complete_description())

In [ ]:
import string

max_number_of_states = 25
common_letters = [4, 3, 2, 1, 0]
unique_letters = [1, 1, 2, 2, 3]
number_of_iter_per_case = 5
number_of_iteration = number_of_iter_per_case * len(common_letters)

# Structure to aggregate the results.
results = []
helper = RunLearnLib()
i = 0

# Build the necessary part of the learnLib project.
helper.compileLearnLib()

while i < number_of_iteration:
    random.seed(i)
    k = common_letters[i % len(common_letters)]  # number of common letters
    l = unique_letters[i % len(unique_letters)]  # number of unique letters
    input_signs = [a for a in string.ascii_lowercase[:l + k + l]]
    i += 1

    # Create two random DFAs and run the learning process to minimize them.
    # Exclude the degenerate cases (DFA of size smaller than 2).
    d1, d2 = DFA(), DFA()
    while d1.Q < 2:
        dfa1 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs[:k + l],
        )
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    while d2.Q < 2:
        dfa2 = DFA()
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs[-(k + l):],
        )
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    conv_d = DFA()
    conv_d.create_convolution_with_common_letters(d1, d2)
    # Save the description of the conv_d automaton.
    conv_d.save_complete_description()

    # Learn conv_d without advice using LearnLib implementation of L*.
    lstar_output = helper.runLstar()
    # Learn conv_d with advice using LearnLib implementation of L*.
    lstar_as_output = helper.runLstarConvWithCLWithAdvice()

    results.append(
        Record(dfa_sz=conv_d.Q,
               lstar=lstar_output,
               lstar_as=lstar_as_output,
               common=k / (l + k),
               target=conv_d))
    print(f"iter {i} done")

In [17]:
results.sort(key=lambda x: (x.common, x.dfa_sz))

In [18]:
lstar_eqs = [r.lstar[1] for r in results]
lstar_as_eqs = [r.lstar_as[1] for r in results]

red = (
    (np.array(lstar_eqs) - np.array(lstar_as_eqs)) / np.array(lstar_eqs)) * 100
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Common letters', ''), ('L*', 'EQ'), ('L* with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.dfa_sz for r in results],
    columns[1]: [int(r.common*100) for r in results],
    columns[2]: lstar_eqs,
    columns[3]: lstar_as_eqs,
    columns[4]: [int(r) for r in red]
}

tables_creator = Tables()
df = tables_creator.create_table_conv_with_cl(columns, data, common_letters, number_of_iter_per_case)
display(df)

,Target language,Common letters,L*,L* with advice,Reduction
,"conv(DFA1, DFA2)",,EQ,EQ,EQ
0,220,0%,59,3,94%
1,247,0%,58,2,96%
2,322,0%,138,7,94%
3,360,0%,97,4,95%
4,396,0%,132,5,96%
5,,,,,
6,180,33%,36,2,94%
7,180,33%,49,3,93%
8,238,33%,29,2,93%
